Importation des modules nécéssaires

In [99]:
import pandas as pd
from langdetect import detect
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("Data_Satisfaction_Original.csv")

In [6]:
#Suppressions colonne maj et Commentaires NAN

In [100]:
df =df.drop(['maj'], axis = 1)
df = df.dropna(axis = 0, how = "all", subset =["Commentaire"])

In [ ]:
#création d'une colonne catégorisant la langue d'un commentaire

In [101]:
df["Language"] = df.Commentaire.apply(lambda row : detect(str(row)))

In [ ]:
#suppressions des lignes de commentaires en langue étrangere

In [102]:
df = df.drop(df[df["Language"]!= "fr"].index, axis = 0)

In [ ]:
#suppression des doublons

In [103]:
df = df.drop_duplicates(keep = "first")

In [ ]:
#Attribution valeur 1 ou 0 suivant la présence de données ou de NAN pour les colonnes client, ville 

In [104]:
df["client"] = df["client"].isna().replace(to_replace =[False,True], value=[1,0])
df["ville"] = df["ville"].isna().replace(to_replace =[False,True], value=[1,0])

Attribution de valeur numérique pour les colonnes source et company

In [105]:
df["source"] = df["source"].isna().replace(to_replace =["TrustPilot","TrustedShop"], value=[1,0]).astype("int64")
df["company"] = df["company"].isna().replace(to_replace =["ShowRoom","VeePee"], value=[1,0]).astype("int64")

Attribution de valeur numérique de sentiment pour la colonne reponse: 0 pour Nan, 1 pour bon, 2 pour mauvais

In [106]:
df["sentiment_reponse"] = df.reponse.apply(lambda row : "absente" if pd.isnull(row)
                                            else ("positive" if ("merci" or "remercions") in row.lower()
                                                       else "negative"))

In [107]:
df = df.join(pd.get_dummies(df["sentiment_reponse"],prefix="reponse", prefix_sep='_'))

remplacement des na de la colonne ecart par la modalité la plus fréquente observée

In [108]:
df["ecart"] = df["ecart"].fillna(df["ecart"].mode()[0])

suppression colonne date_commande

In [109]:
df =df.drop(["date_commande"], axis = 1)

Transformation date

In [110]:
df["date"]=pd.to_datetime(df['date'], utc=True)

In [111]:
df["date_annee"] = pd.DatetimeIndex(df['date']).year
df["date_mois"] = pd.DatetimeIndex(df['date']).month
df["date_jour"]= pd.DatetimeIndex(df['date']).day


Remplacement des dates NAN par le mode

In [112]:
df["date_annee"] = df["date_annee"].fillna(df["date_annee"].mode()[0]) 
df["date_mois"] = df["date_mois"].fillna(df["date_mois"].mode()[0])
df["date_jour"] = df["date_jour"].fillna(df["date_jour"].mode()[0])

normalisation des valeurs numérique supérieur à 1

In [114]:
mmscaler = MinMaxScaler()

In [159]:
df[["ecart","date_annee","date_mois","date_jour"]] = mmscaler.fit_transform(df[["ecart","date_annee","date_mois","date_jour"]])

In [164]:
df2 =df.drop(["date",'reponse',"Language","sentiment_reponse"], axis = 1)

,Commentaire,star,client,source,company,ville,ecart,reponse_absente,reponse_negative,reponse_positive,date_annee,date_mois,date_jour
0,"Bonjour , Ca doit faire 5 ans environ que je s...",1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.633333
1,Vente lacoste article manquant photo prise sur...,1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.633333
2,"Vente Lacoste Honteuse , article erroné , arti...",1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.600000
3,J'ai commandé des mules de la marque Moosefiel...,2,1,0,0,0,0.06993,0,1,0,1.0,0.454545,0.600000
4,Commande téléphone etat A+ . Livraison d un vi...,1,1,0,0,0,0.06993,0,1,0,1.0,0.454545,0.600000


Mise à jour du stop word et application aux commentaires.

In [168]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

stop_words.update(["toujour", "commande","article","euro","faire","donc","car"," si ","c'est","comme"," alor "," fait "])
stop_words.update([".",",","!","'","...","’", "a","toujours","tres","j'ai","ça","chez","quand"])

In [170]:
def stop_words_filtering(my_comment):
    new_comment =""
    for word in my_comment.lower().split(" "):
        if word not in stop_words:
            new_comment += word
            new_comment += " "
    return new_comment

df2["Commentaires_reduit"] = df2.Commentaire.apply(lambda row : stop_words_filtering(row))

Suppression des nombres ou des numero dans les commentaires

In [173]:
import re
r = re.compile(r"[0-9]+") 
def words_nonum(my_comment):
    new_comment =r.sub('', my_comment)
    return new_comment

c'est super cool comme superstition


In [177]:
df2["Commentaires_reduit"] = df.Commentaires_reduit.apply(lambda row : words_nonum(row))

In [180]:
df2 = df2.drop("Commentaire",axis=1) 

In [181]:
df2.head()

,star,client,source,company,ville,ecart,reponse_absente,reponse_negative,reponse_positive,date_annee,date_mois,date_jour,Commentaires_reduit
0,1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.633333,bonjour ca doit ans environ membre showroopri...
1,1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.633333,vente lacoste manquant photo prise articles m...
2,1,1,0,0,0,0.06993,1,0,0,1.0,0.454545,0.600000,vente lacoste honteuse erroné manquant bon liv...
3,2,1,0,0,0,0.06993,0,1,0,1.0,0.454545,0.600000,commandé mules marque moosefield très déçue pr...
4,1,1,0,0,0,0.06993,0,1,0,1.0,0.454545,0.600000,téléphone etat a+ livraison vieux téléphone po...


In [186]:
df2.isna().sum()

star                   0
client                 0
source                 0
company                0
ville                  0
ecart                  0
reponse_absente        0
reponse_negative       0
reponse_positive       0
date_annee             0
date_mois              0
date_jour              0
Commentaires_reduit    0
dtype: int64

In [184]:
df2.to_csv("Data_Satisfaction_retraitement2.csv")

In [3]:
#all in one 
import pandas as pd
from langdetect import detect
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("Data_Satisfaction_Original.csv")
df =df.drop(['maj'], axis = 1)
df = df.dropna(axis = 0, how = "all", subset =["Commentaire"])
df["Language"] = df.Commentaire.apply(lambda row : detect(str(row)))
df = df.drop(df[df["Language"]!= "fr"].index, axis = 0)
df = df.drop_duplicates(keep = "first")
df["client"] = df["client"].isna().replace(to_replace =[False,True], value=[1,0])
df["ville"] = df["ville"].isna().replace(to_replace =[False,True], value=[1,0])
df["source"] = df["source"].isna().replace(to_replace =["TrustPilot","TrustedShop"], value=[1,0]).astype("int64")
df["company"] = df["company"].isna().replace(to_replace =["ShowRoom","VeePee"], value=[1,0]).astype("int64")
df["sentiment_reponse"] = df.reponse.apply(lambda row : "absente" if pd.isnull(row)
                                            else ("positive" if ("merci" or "remercions") in row.lower()
                                                       else "negative"))
df = df.join(pd.get_dummies(df["sentiment_reponse"],prefix="reponse", prefix_sep='_'))
df["ecart"] = df["ecart"].fillna(df["ecart"].mode()[0])
df =df.drop(["date_commande"], axis = 1)

df["date"]=pd.to_datetime(df['date'], utc=True)
df["date_annee"] = pd.DatetimeIndex(df['date']).year
df["date_mois"] = pd.DatetimeIndex(df['date']).month
df["date_jour"]= pd.DatetimeIndex(df['date']).day
df["date_annee"] = df["date_annee"].fillna(df["date_annee"].mode()[0]) 
df["date_mois"] = df["date_mois"].fillna(df["date_mois"].mode()[0])
df["date_jour"] = df["date_jour"].fillna(df["date_jour"].mode()[0])
mmscaler = MinMaxScaler()
df[["ecart","date_annee","date_mois","date_jour"]] = mmscaler.fit_transform(df[["ecart","date_annee","date_mois","date_jour"]])
df2 =df.drop(["date",'reponse',"Language","sentiment_reponse"], axis = 1)

from nltk.corpus import stopwords
stop_words = set(stopwords.words('french'))

stop_words.update(["toujour","faire","donc","car",
                   " si ","c'est","comme"," alor","fait", "a","toujours","tres",
                   "j'ai","ça","chez","quand","très","tout","fois", "alors", "avoir","rien"])



def stop_words_filtering(my_comment):
    new_comment =""
    for word in my_comment.lower().split(" "):
        if word not in stop_words:
            new_comment += word
            new_comment += " "
    return new_comment



import re
r = re.compile(r"[0-9]+") 
def words_nonum(my_comment):
    new_comment =r.sub('', my_comment)
    new_comment2 = re.sub(r'[^\w\s]', ' ', new_comment)
    return new_comment2
df2["Commentaires_reduit"] = df2.Commentaire.apply(lambda row : words_nonum(row))

stop_words.update([" l ", "d "," n ", " j ", " qu ", " m "])

df2["Commentaires_reduit"] = df2.Commentaires_reduit.apply(lambda row : stop_words_filtering(row))
df2 = df2.drop("Commentaire",axis=1) 

df2.to_csv("Data_Satisfaction_retraitement4.csv",index=False)